In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(100)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re
import html as html_lib

import json

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/3.model.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Year'] = df_input['Year'].astype(int)
df_input['Make No'] = df_input['Make No'].astype(int)
df_input['Model No'] = df_input['Model No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        data = {'search_url': 'https://www.rainx.com/',
                'bf-year': crawler_series['Year'],
                'bf-make': crawler_series['Make'],
                'bf-model': crawler_series['Model']}

        count_retry = 0
        while True:
            count_retry += 1

            try:
                url_check = 'https://www.rainx.com/'

                resp = requests.post('https://www.rainx.com/blade-size-finder/',
                                     data=data,
                                     headers=get_header(),
                                     proxies=get_proxy(),
                                     timeout=(5, 60))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    if html.xpath('//div[@class="your_vechicle clearfix"]/label/strong/text()') and html.xpath('//div[@class="your_vechicle clearfix"]/label/strong/text()')[0] == f'''{crawler_series['Year']} {crawler_series['Make']} {crawler_series['Model']}''' or html.xpath('//div[@class="no-result"]/h2/text()') and html.xpath('//div[@class="no-result"]/h2/text()')[0] == 'We\'re Sorry...':
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if html.xpath('//div[@class="no-result"]/h2/text()') and html.xpath('//div[@class="no-result"]/h2/text()')[0] == 'We\'re Sorry...':
            raise

        # = = = = = = = = = = = = = = =

        list_dict_correct_temp, list_div = [], html.xpath('//div[@class="blade-size-finder-container"]/div[@class="col-3"]')
        for i, div in enumerate(list_div):
            category, category_description = div.xpath('./header/a/h3/text()')[0].strip(), div.xpath('./header/a/span[@class="h6"]/text()')[0].strip()

            list_li = div.xpath('./ul/li')
            for j, li in enumerate(list_li):
                status = li.xpath('./@class')[0].strip()

                url = li.xpath('./a/@href')[0].strip()

                dict_src, list_src = {}, li.xpath('./a/figure/img/@src')
                for k, src in enumerate(list_src):
                    dict_src[str(k)] = src.strip()

                span = li.xpath('./a/span[@class="h6 wiper_blade_heading"]')[0]
                title = etree.tostring(span).decode('utf-8')
                title = re.sub(r'<.*?>', '', title)
                title = re.sub(r' +', ' ', title.strip())
                title = html_lib.unescape(title)

                dict_correct_temp = {'Year': crawler_series['Year'],
                                     'Make No': crawler_series['Make No'],
                                     'Make': crawler_series['Make'],
                                     'Model No': crawler_series['Model No'],
                                     'Model': crawler_series['Model'],
                                     'Category No': i + 1,
                                     'Category': category,
                                     'Category Description': category_description,
                                     'Part No': j + 1,
                                     'Status': status,
                                     'Title': title,
                                     'Picture': '',
                                     'Url': url,
                                     'Json_Src': json.dumps(dict_src)}

                dict_attr, list_key, list_value = {},li.xpath('./div[@class="product_blade_size clearfix"]/ul[@class="product_size_labels"]/li/strong/text()'),li.xpath('./div[@class="product_blade_size clearfix"]/ul[@class="product_size_labels"]/li/span/text()')
                for k, (key, value) in enumerate(zip(list_key, list_value)):
                    dict_attr[str(k)] = {key.strip(): value.strip()}

                list_head, list_row = li.xpath('./div[@class="product_blade_size clearfix"]/ul[@class="product_size_labels"]/li[1]/strong/text()'), li.xpath('./div[@class="product_blade_size clearfix"]/ul[@class="product_size_labels"]/li')[1:]
                for k, row in enumerate(list_row):
                    for m, head in enumerate(list_head):
                        if k == 0:
                            dict_correct_temp[head] = row.xpath(f'./span[{m+1}]/text()')[0].strip()
                        else:
                            dict_correct_temp[f'{head}_{k+1}'] = row.xpath(f'./span[{m+1}]/text()')[0].strip()

                list_dict_correct_temp.append(dict_correct_temp)

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.extend(list_dict_correct_temp)

        # = = = = = = = = = = = = = = =

        crawler_status = True
    except:
        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['Year']} -> {crawler_series['Make No']}. {crawler_series['Make']} -> {crawler_series['Model No']}. {crawler_series['Model']}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Year', 'Make No', 'Model No', 'Category No', 'Part No'],
                                                             ascending=[False, True, True, True, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/4.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Year', 'Make No', 'Model No'],
                                                           ascending=[False, True, True],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/part_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[True] - [请求1次] - [剩余299条] - [2024/11/20 15:46:35] - 1970 -> 3. Buick -> 2. Estate Wagon

[True] - [请求1次] - [剩余298条] - [2024/11/20 15:46:37] - 1970 -> 3. Buick -> 3. LeSabre

[True] - [请求1次] - [剩余297条] - [2024/11/20 15:46:37] - 1970 -> 3. Buick -> 4. Riviera

[True] - [请求1次] - [剩余296条] - [2024/11/20 15:46:39] - 1970 -> 3. Buick -> 5. Wildcat

[True] - [请求1次] - [剩余295条] - [2024/11/20 15:46:41] - 1970 -> 4. Cadillac -> 1. Calais

[True] - [请求1次] - [剩余294条] - [2024/11/20 15:46:42] - 1970 -> 4. Cadillac -> 3. Eldorado

[True] - [请求1次] - [剩余293条] - [2024/11/20 15:46:44] - 1970 -> 5. Chevrolet -> 4. G10 Van

[True] - [请求1次] - [剩余292条] - [2024/11/20 15:46:44] - 1970 -> 4. Cadillac -> 4. Fleetwood

[True] - [请求1次] - [剩余291条] - [2024/11/20 15:46:45] - 1970 -> 5. Chevrolet -> 2. Biscayne

[True] - [请求1次] - [剩余290条] - [2024/11/20 15:46:45] - 1970 -> 5. Chevrolet -> 1. Bel Air

[True] - [请求1次] - [剩余289条] - [2024/11/20 15:46:45] - 1970 -> 5. Chevrolet -> 7. P10 Van

[True] - [请求1次] - [剩余288条] - [20